# Instrument Classification - Part 3 : Track Prediction

In [1]:
import numpy as np
import librosa, os, keras, time
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.utils import make_chunks

Using TensorFlow backend.


In [2]:
def crop_song(dir,songs):
    predict = []
    pred_all = []
    even = True
    it = 0
    for f in os.listdir(dir):
        if (it == songs): break
        if (even): 
            even = False 
            continue
        else:
            print(f)
            even = True
            # downsampled to 22050Hz and downmixed to mono
            wav,sr = librosa.load(dir+f,mono=True,sr=22050)
            # normalized by the root mean square energy
            wav = wav / np.sqrt(np.mean(wav**2))
            # chunked to 1sec long snippets
            chunks = make_chunks(wav,sr)
            for wav in chunks:
                # transformed into mel-spectrograms with given traits
                mel = librosa.feature.melspectrogram(wav,n_mels=96,fmax=11025,n_fft=1024,hop_length=256,power=1)
                # decibel scaling
                mel = librosa.core.amplitude_to_db(mel)
                # zero padding when mel shape is not (96,87)
                pad = np.zeros((96,87))
                if (mel.shape != pad.shape):
                    pad[:mel.shape[0],:mel.shape[1]] = mel
                    mel = pad
                predict.append(mel)
        it = it + 1
        pred_all.append(predict)
        predict = []
    return pred_all

def decide(pred):
    pred_m = []
    pred_avg = []
    for song in pred:
        maxp, avgp = [0,0,0,0], [0,0,0,0]
        for tupl in song:
            for i in range(len(tupl)):
                if tupl[i]>maxp[i]: maxp[i] = tupl[i]
                avgp[i] += tupl[i]/len(song)
        #for i in range(4): avgp[i] = avgp[i]/4        
        pred_m.append(maxp)
        pred_avg.append(avgp)
    return pred_m, pred_avg
        
#def extract_instruments(prediction):
    # connect to_categorical to instrument labels

In [62]:
model3 = keras.models.load_model('ICmodel_4_mix_big_more.h5')
model1 = keras.models.load_model('ICmodel_2_mix_big_more.h5')

In [8]:
dir = './IRMAS/TestingData/Part2/'
songs = 3
predict = crop_song(dir,songs)

000015_more than a feeling - boston-1.wav
000015_more than a feeling - boston-2.wav
000015_more than a feeling - boston-3.wav


In [10]:
pred_all = []
for song in predict:
    new = np.expand_dims(np.array(song),-1)
    prediction = model1.predict(new)
    pred_all.append(prediction)
n, m = decide(pred_all)

print("Evaluation for gac-voi improved mix model")
print("Song with gac")
print("max:",n[1])
print("avg:",m[1])
print("Song with gac-voi")
print("max:",n[2])
print("avg:",m[2])

Evaluation for gac-voi improved mix model
Song with gac
max: [0.17979953, 0.99986196, 0.4000203, 0.72833025]
avg: [0.03728673926421574, 0.9740035533905028, 0.06954999055181231, 0.45899773921285353]
Song with gac-voi
max: [0.6526026, 0.9965706, 0.68033624, 0.9665273]
avg: [0.13124338388442994, 0.8770530432462693, 0.1544665530323982, 0.6500529810786247]


In [73]:
dir = './IRMAS/TestingData/YT/'
predict = crop_song(dir,1)
pred_all = []
for song in predict:
    new = np.expand_dims(np.array(song),-1)
    prediction = model3.predict(new)
    pred_all.append(prediction)
n, m = decide(pred_all)

nanoi.wav


In [74]:
pia_thr = 0.5
gac_thr = 0.5
dru_thr = 0.5
voi_thr = 0.5

instr = ['pia','gac','dru','voi']
thr = [pia_thr,gac_thr,dru_thr,voi_thr]
new_prediction = [[0]*len(instr)]*(len(prediction))
bool = [False,False,False,False]

#instr = ['gac','voi']
#thr = [gac_thr,voi_thr]
#bool = [False,False]

new_prediction = []
for i in range(len(prediction)):
    y = []
    for j in range(0,len(instr)):
        y.append(0)
    new_prediction.append(y)

for i in range(len(prediction)):
    for j in range(len(instr)):
        if i==0:
            if (prediction[i][j] < thr[j] and prediction[i+1][j] > thr[j]): new_prediction[i][j] = prediction[i+1][j]
            elif (prediction[i][j] > thr[j] and prediction[i+1][j] < thr[j]): new_prediction[i][j] = prediction[i+1][j]
            else: new_prediction[i][j] = prediction[i][j]
        elif i==len(prediction)-1:
            if (prediction[i][j] < thr[j] and prediction[i-1][j] > thr[j]): new_prediction[i][j] = prediction[i-1][j]
            elif (prediction[i][j] > thr[j] and prediction[i-1][j] < thr[j]): new_prediction[i][j] = prediction[i-1][j]
            else: new_prediction[i][j] = prediction[i][j]
        else:
            if (prediction[i][j]<thr[j] and prediction[i-1][j]>thr[j] and prediction[i+1][j]>thr[j]):
                new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
            elif (prediction[i][j]>thr[j] and prediction[i-1][j]<thr[j] and prediction[i+1][j]<thr[j]):
                new_prediction[i][j] = (prediction[i-1][j]+prediction[i+1][j])/2
            else: new_prediction[i][j] = prediction[i][j]
        if (new_prediction[i][j]>thr[j]): bool[j] = True
            
            
print("The track contains: ",end=" ")
for i in range(len(bool)):
    if (bool[i]): print(instr[i],end=" ")

print("\nBegins in 5 seconds")
time.sleep(1)
print("4")
time.sleep(1)
print("3")
time.sleep(1)
print("2")
time.sleep(1)
print("1")
time.sleep(1)
print("Go!")
for i in range(len(prediction)):
    time.sleep(1)  
    print(str(i)+"-"+str(i+1)+" sec: ", end =" ")
    for j in range(len(instr)):
        if (new_prediction[i][j]>thr[j]): print(instr[j],round(new_prediction[i][j],2), end =" ")
    print("")

#t = np.arange(0,len(prediction))
#plt.figure(figsize=(50,10))
#plt.plot(t[:150],new_prediction[:150][0],new_prediction[:150][1])

The track contains:  pia gac dru voi 
Begins in 5 seconds
4
3
2
1
Go!
0-1 sec:  pia 0.55 voi 0.82 
1-2 sec:  pia 0.86 voi 0.71 
2-3 sec:  pia 0.94 gac 0.99 
3-4 sec:  pia 0.79 gac 0.99 
4-5 sec:  pia 0.81 gac 0.99 
5-6 sec:  pia 0.68 gac 0.8 
6-7 sec:  pia 0.82 voi 0.96 
7-8 sec:  pia 0.64 voi 0.64 
8-9 sec:  pia 0.85 voi 0.55 
9-10 sec:  pia 0.69 voi 0.98 
10-11 sec:  pia 0.83 gac 0.98 
11-12 sec:  pia 0.97 gac 0.88 
12-13 sec:  pia 0.98 gac 0.94 
13-14 sec:  pia 0.95 gac 0.95 
14-15 sec:  pia 0.96 gac 1.0 
15-16 sec:  pia 0.93 gac 0.96 
16-17 sec:  
17-18 sec:  dru 0.52 voi 0.91 
18-19 sec:  voi 0.88 
19-20 sec:  pia 0.83 voi 0.73 
20-21 sec:  pia 0.69 voi 0.81 
21-22 sec:  pia 0.7 gac 1.0 
22-23 sec:  pia 0.72 gac 0.99 
23-24 sec:  pia 0.51 gac 0.99 
24-25 sec:  pia 0.74 gac 1.0 
25-26 sec:  pia 0.81 gac 0.99 
26-27 sec:  pia 0.96 
27-28 sec:  voi 0.88 
28-29 sec:  voi 0.97 
29-30 sec:  voi 0.86 
30-31 sec:  voi 0.82 
31-32 sec:  pia 0.88 gac 0.95 
32-33 sec:  pia 0.85 gac 0.99 
33-

In [65]:
new_prediction[49:55]

[[0.7943812, 0.37397492],
 [0.8117258548736572, 0.7657894],
 [0.33893829584121704, 0.87507164],
 [0.9084110856056213, 0.8977815],
 [0.4630456268787384, 0.9065616130828857],
 [0.8048422336578369, 0.91534173]]